Folder Path and libraries

In [7]:
import pandas as pd
import re
import os
import pandas as pd
import datetime

#CONST

folder_path = r"D:\data\CGIAR\UNFCCC"

1. Adaptation Fund

Add the project number to the original CSV file downloaded from the 'adaptation_fund_path' website. Also, identify projects listed on the website that have no corresponding documents in 'AF_projects_not_in_documents'. Date files indicate when the data collection. 


In [2]:
import pandas as pd
import datetime

# Paths to your CSV files

adaptation_fund_path = os.path.join(folder_path, "processing_stage_movebackmain", "AF_20240403.csv")  #the file downloaded as csv from ADF
project_documents_path = os.path.join(folder_path, "processing_stage_movebackmain", "AF_project_documents_links_with_titles_unique.csv")



# Read the CSV files
df_adaptation_fund = pd.read_csv(adaptation_fund_path, on_bad_lines='skip',encoding='utf-8')
df_project_documents = pd.read_csv(project_documents_path, on_bad_lines='skip',encoding='utf-8')

# Ensure 'project_link' is in df_project_documents to proceed with the merge
df_joined = pd.merge(df_adaptation_fund, df_project_documents[['project_link', 'project_title', 'document_link', 'project_no']], left_on='projecturl', right_on='project_link', how='left')

    # Remove the 'project_link' column
df_joined.drop('project_link', axis=1, inplace=True)


current_time = datetime.datetime.now().strftime("%Y%m%d")
#find if projects no documents
projects_not_in_documents = df_adaptation_fund[~df_adaptation_fund['projecturl'].isin(df_project_documents['project_link'])]
filename = f"AF_projects_not_in_documents_{current_time}.csv"
projects_not_in_documents.to_csv(rf"C:\Users\david\My Drive\data\analysis_git_data\cgiar\{filename}", index=False)

# drop if no project number

df_joined_notna = df_joined[df_joined['project_no'].notnull()]
#add org column for master file
df_joined_notna['org'] = 'AF'
df_joined_notna.rename(columns={'projecturl': 'url', 'grantamount':'grant_amount', 'projecttitle': 'title', 'ie':'entity', 'fy_approval':'year'}, inplace=True) #rename for consistency with master // TODO:check with Cesare if BM is status.  

df_joined_notna['theme'] = 'Adaptation'

current_time = datetime.datetime.now().strftime("%Y%m%d")
filename = f"AF_joined_projects_{current_time}.csv"

full_path = os.path.join(folder_path, filename)

df_joined_notna.to_csv(full_path, index=False,encoding='utf-8')





OSError: Cannot save file into a non-existent directory: 'C:\Users\david\My Drive\data\analysis_git_data\cgiar'

2. GCF 

In [ ]:


gcf_projects = pd.read_csv(os.path.join(folder_path, "processing_stage_movebackmain", "gcf_projects_20240403.csv"),encoding='utf-8') #the file downloaded as csv from GCF
gcf_projects['org'] = 'GCF'
gcf_projects.rename(columns={'Ref #': 'project_no', 'Countries': 'country', 'Project Name': 'title', 'Sector': 'sector', 'Entity': 'entity', 'BM': 'year', 'FA Financing': 'grant_amount', 'Theme': 'theme'}, inplace=True) #rename for consistency with master // TODO:check with Cesare if BM is status.  


current_time = datetime.datetime.now().strftime("%Y%m%d")
filename = f"GCF_joined_projects_{current_time}.csv"
file_path = os.path.join(folder_path, filename)

gcf_projects.to_csv(file_path, index=False,encoding='utf-8')


3. GEF

In [ ]:
gef_df = pd.read_csv(os.path.join(folder_path, "processing_stage_movebackmain", "gef_projects.csv"),encoding='utf-8') #the file downloaded as csv from GEF with filter climate change
gef_df['org'] = 'GEF'
gef_df.rename(columns={'ID': 'project_no', 'Countries': 'country', 'Title': 'title', 'Focal Areas': 'sector', 'Agencies': 'entity', 'GEF Grant': 'grant_amount', 'Status':'status', 'Approval FY': 'year'}, inplace=True) #rename for consistency with master

base_url = 'https://www.thegef.org/projects-operations/projects/'

gef_df['url'] = gef_df['project_no'].apply(lambda x: base_url + str(x))


gef_df['theme'] = 'NA'



current_time = datetime.datetime.now().strftime("%Y%m%d")
filename = f"GEF_joined_projects_{current_time}.csv"
file_path = os.path.join(folder_path, filename)
gef_df.to_csv(file_path, index=False,encoding='utf-8')


Creation of the Master file

In [48]:
import pandas as pd
import pycountry
from unidecode import unidecode
import re
import datetime
import os

# Load the CSV file with UTF-8 encoding
data = pd.read_csv(r"D:\data\CGIAR\UNFCCC\processing_stage_movebackmain\master_file_af_gcf_gef_20240517.csv", encoding='utf-8')

# Extended mapping for normalizing country names and their ISO3 codes
country_normalization_mapping = {
    'Libyan Arab Jamahiriya': 'Libya',
    'Côte d’Ivoire': 'CIV',
    'Cote d\'Ivoire': 'CIV',
    'C&ocirc;te d\'Ivoire': 'CIV',
    'Micronesia, Federated States of': 'FSM',
    'Micronesia (Federated States of)': 'FSM',
    'Micronesia': 'FSM',
    'Comoros (the)': 'Comoros',
    'Democratic Republic of the Congo (the)': 'COD',
    'Lao People\'s Democratic Republic (the)': 'LAO',
    'State of Palestine': 'PSE',
    'Niger (the)': 'NER',
    'Philippines (the)': 'PHL',
    'Bolivia (Plurinational State of)': 'BOL',
    'Lao PDR': 'LAO',
    'Timor Leste': 'TLS',
    'Congo DR': 'COD',
    'Bosnia-Herzegovina': 'BIH',
    'Africa, Regional': 'AFR',
    'St. Kitts And Nevis': 'KNA',
    'St. Lucia': 'LCA',
    'St. Vincent and Grenadines': 'VCT',
    'Korea DPR': 'PRK',
    'Regional, Timor Leste': 'Regional, Timor-Leste',
    'Regional, Bosnia-Herzegovina': 'Regional, Bosnia and Herzegovina',
    'Regional, Micronesia': 'Regional, Micronesia (Federated States of)',
    'Regional, Cote d\'Ivoire': 'Regional, CIV',
    'Regional, St. Lucia': 'Regional, Saint Lucia',
    'Cote d\'Ivoire, Global': 'CIV, Global',
    'Turkiye': 'TUR',
    'Turkey': 'TUR',
    'Central African Republic': 'CAF',
    'Central African Republic (the)': 'CAF',
    'CAR': 'CAF',
    'Dominican Republic': 'DOM',
    'Dominican Republic (the)': 'DOM',
    'Republica Dominicana': 'DOM',
    'Eswatini': 'SWZ',
    'Swaziland': 'SWZ'
}

# Normalize country names
def normalize_country_name(country_name):
    # Handle prefixes like 'Regional,'
    prefixes = ['Regional,', 'Global,']
    for prefix in prefixes:
        if country_name.startswith(prefix):
            base_name = country_name[len(prefix):].strip()
            normalized_base_name = country_normalization_mapping.get(base_name, base_name)
            return f'{prefix} {normalized_base_name}'
    
    # Handle multiple countries
    if ',' in country_name:
        countries = country_name.split(',')
        normalized_countries = [country_normalization_mapping.get(country.strip(), country.strip()) for country in countries]
        return ', '.join(normalized_countries)
    
    # Single country
    normalized_name = country_normalization_mapping.get(country_name, country_name)
    return unidecode(normalized_name)

# Apply the normalization function to the 'country' column
data['country'] = data['country'].apply(normalize_country_name)

# Function to get ISO3 code
def get_iso3(country_name):
    if country_name in country_normalization_mapping:
        return country_normalization_mapping[country_name]
    try:
        return pycountry.countries.lookup(country_name).alpha_3
    except LookupError:
        return 'Unspecified'

# Handle multiple country names in a cell
def convert_countries_to_iso3(countries):
    if ',' in countries:
        # Split the countries and convert each one
        country_list = countries.split(',')
        iso3_list = [get_iso3(country.strip()) for country in country_list]
        return ', '.join([code for code in iso3_list if code])
    else:
        # Single country
        return get_iso3(countries)

data['country_iso3'] = data['country'].apply(convert_countries_to_iso3)

# Counting 'Unspecified' values in the 'country_iso3' column
unspecified_count = (data['country_iso3'] == 'Unspecified').sum()
print("Count of 'Unspecified' in 'country_iso3':", unspecified_count)

# Function to extract country names and convert to ISO3
def extract_countries_to_iso3(title):
    # Patterns to extract country names
    pattern = r'\b(?:[A-Z][a-z]+(?:\s+[A-Z][a-z]+)*)\b'
    
    matches = re.findall(pattern, title)
    
    # Filter out words that are not country names
    country_names = []
    for name in matches:
        try:
            country = pycountry.countries.lookup(name)
            country_names.append(country.alpha_3)
        except LookupError:
            continue
    
    # Join the ISO3 codes as a comma-separated string
    return ', '.join(country_names) if country_names else 'Unspecified'

filtered_df = data[data['country_iso3'] == 'Unspecified']
# Apply the function to the entire DataFrame
filtered_df['country_iso3'] = filtered_df['title'].apply(extract_countries_to_iso3)

# Merge filtered_df back to the original data DataFrame
data.update(filtered_df[['country_iso3']])

# Counting 'Unspecified' values in the 'country_iso3' column again
unspecified_count = (data['country_iso3'] == 'Unspecified').sum()
print("Count of 'Unspecified' in 'country_iso3':", unspecified_count)

current_time = datetime.datetime.now().strftime("%Y%m%d")
data.to_csv("D:\\data\\CGIAR\\UNFCCC\\master.csv", index=False, encoding='utf-8')

# Check for the files
folder_path = "D:\\data\\CGIAR\\UNFCCC"
af_df = pd.read_csv(os.path.join(folder_path, "AF_joined_projects_20240517.csv"), encoding='utf-8')
gcf_df = pd.read_csv(os.path.join(folder_path, "GCF_joined_projects_20240517.csv"), encoding='utf-8')
gef_df = pd.read_csv(os.path.join(folder_path, "GEF_joined_projects_20240517.csv"), encoding='utf-8')

print(f"Total projects from the Adaptation Fund are {len(af_df)}. On the website, there are 149. Missing or not in dataset: {149 - len(af_df)}.")
print(f"Total projects from the GCF are {len(gcf_df)}. On the website, there are 253. Missing or not in dataset: {253 - len(gcf_df)}.")
print(f"Total projects from the GEF are {len(gef_df)}. On the website, there are 2402. Missing or not in dataset: {2402 - len(gef_df)}.")


Count of 'Unspecified' in 'country_iso3': 139
Count of 'Unspecified' in 'country_iso3': 114
Total projects from the Adaptation Fund are 147. On the website, there are 149. Missing or not in dataset: 2.
Total projects from the GCF are 253. On the website, there are 253. Missing or not in dataset: 0.
Total projects from the GEF are 2402. On the website, there are 2402. Missing or not in dataset: 0.


C:\Users\david\AppData\Local\Temp\ipykernel_19372\1467963200.py:124: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['country_iso3'] = filtered_df['title'].apply(extract_countries_to_iso3)


Explode master file

In [49]:
import pandas as pd
import pycountry

# URL of the GitHub raw file
GITHUB_URL = 'https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/master/all/all.csv'

# Load the CSV file directly from GitHub
regional_data = pd.read_csv(GITHUB_URL)

# Load the master CSV file
FILE_PATH = r"D:\data\CGIAR\UNFCCC\master.csv"
data = pd.read_csv(FILE_PATH, encoding='utf-8')

# Ensure 'Unspecified' entries are excluded
data = data[data['country_iso3'].str.strip().str.lower() != 'unspecified']

# Remove commas from 'grant_amount' and convert to numeric (float)
data['grant_amount'] = data['grant_amount'].str.replace(',', '', regex=False).astype(float)

# Function to normalize country names to ISO3 codes
def normalize_country_name(country_name):
    try:
        return pycountry.countries.lookup(country_name).alpha_3
    except LookupError:
        return None

# Function to divide regional multinational projects
def divide_multinational_projects(row):
    countries = row['country'].split(', ')
    country_iso3_list = row['country_iso3'].split(', ')
    grant_amount = row['grant_amount'] / len(countries)
    rows = []
    for country, iso3 in zip(countries, country_iso3_list):
        new_row = row.copy()
        new_row['country'] = country
        new_row['country_iso3'] = iso3
        new_row['grant_amount'] = grant_amount
        rows.append(new_row)
    return rows

# Apply the function to create individual rows for each country in multinational projects
expanded_data = data.apply(lambda row: divide_multinational_projects(row), axis=1)
expanded_data = [item for sublist in expanded_data for item in sublist]
expanded_data = pd.DataFrame(expanded_data)

# Conflict-affected and Fragile countries from World Bank
CONFLICT_COUNTRIES = [
    'AFG', 'BFA', 'CMR', 'CAF', 'COD', 
    'ETH', 'IRQ', 'MLI', 'MOZ', 'MMR', 'NER', 'NGA', 'SOM', 
    'SSD', 'SDN', 'SYR', 'UKR', 'PSE', 'YEM'
]

FRAGILE_COUNTRIES = [
    'BDI', 'TCD', 'COM', 'COG', 'ERI', 'GNB', 'HTI', 
    'KIR', 'XKX', 'LBN', 'LBY', 'MHL', 'FSM', 
    'PNG', 'STP', 'SLB', 'TLS', 'TUV', 
    'VEN', 'ZWE'
]

conflict_iso3 = [normalize_country_name(country) for country in CONFLICT_COUNTRIES]
fragile_iso3 = [normalize_country_name(country) for country in FRAGILE_COUNTRIES]

# Remove None values
conflict_iso3 = [iso3 for iso3 in conflict_iso3 if iso3]
fragile_iso3 = [iso3 for iso3 in fragile_iso3 if iso3]

# Add 'FCS', 'FCS_conflict', and 'FCS_fragile' columns
expanded_data['FCS'] = expanded_data['country_iso3'].isin(conflict_iso3 + fragile_iso3)
expanded_data['FCS_conflict'] = expanded_data['country_iso3'].isin(conflict_iso3)
expanded_data['FCS_fragile'] = expanded_data['country_iso3'].isin(fragile_iso3)

# Map countries to their sub-regions using the regional data
regional_data_mapping = dict(zip(regional_data['alpha-3'], regional_data['sub-region']))
expanded_data['sub-region'] = expanded_data['country_iso3'].map(regional_data_mapping)

# Exclude rows with 'country_iso3' as 'Unspecified'
expanded_data = expanded_data[expanded_data['country_iso3'] != 'Unspecified']

# Save the expanded data to a new CSV file
EXPANDED_FILE_PATH = r"D:\data\CGIAR\UNFCCC\exploded_master.csv"
expanded_data.to_csv(EXPANDED_FILE_PATH, index=False, encoding='utf-8')


2nd master file pop aggregate

In [61]:
import pandas as pd
import pycountry
import requests

# Load the exploded master file
FILE_PATH = r'D:\data\CGIAR\UNFCCC\exploded_master.csv'
data = pd.read_csv(FILE_PATH, encoding='utf-8')

# Group by country and sum the total grant amount per country
grouped_data = data.groupby('country_iso3')['grant_amount'].sum().reset_index()

# Function to get population data from World Bank API
def get_population_data(iso3):
    url = f"https://api.worldbank.org/v2/country/{iso3}/indicator/SP.POP.TOTL?date=2022&format=json"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        if len(data) > 1 and isinstance(data[1], list) and len(data[1]) > 0:
            return data[1][0].get('value', 'Unknown')
    except requests.exceptions.RequestException as e:
        print(f"HTTP error: {e}")
    except (IndexError, KeyError, TypeError) as e:
        print(f"Data error: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")
    
    return 'Unknown'

# Add population data to the grouped data
grouped_data['population'] = grouped_data['country_iso3'].apply(get_population_data)

# Calculate per capita grant amount
grouped_data['grant_per_capita'] = grouped_data.apply(
    lambda row: row['grant_amount'] / row['population'] if row['population'] != 'Unknown' else 'Unknown', axis=1
)

# Group by country and organization, and sum the grant amount per country and organization
org_grouped_data = data.groupby(['country_iso3', 'org'])['grant_amount'].sum().unstack(fill_value=0).reset_index()

# Calculate per capita grant amount for each organization
for org in org_grouped_data.columns[1:]:
    org_grouped_data[f'grant_per_capita_{org}'] = org_grouped_data.apply(
        lambda row: row[org] / grouped_data.loc[grouped_data['country_iso3'] == row['country_iso3'], 'population'].values[0]
        if grouped_data.loc[grouped_data['country_iso3'] == row['country_iso3'], 'population'].values[0] != 'Unknown'
        else 'Unknown', axis=1
    )

# Normalize country names
def normalize_country_name(country_name):
    try:
        return pycountry.countries.lookup(country_name).alpha_3
    except LookupError:
        return None

# Assuming these variables are defined earlier in the code and not repeated
CONFLICT_COUNTRIES = [
    'AFG', 'BFA', 'CMR', 'CAF', 'COD', 
    'ETH', 'IRQ', 'MLI', 'MOZ', 'MMR', 'NER', 'NGA', 'SOM', 
    'SSD', 'SDN', 'SYR', 'UKR', 'PSE', 'YEM'
]

FRAGILE_COUNTRIES = [
    'BDI', 'TCD', 'COM', 'COG', 'ERI', 'GNB', 'HTI', 
    'KIR', 'XKX', 'LBN', 'LBY', 'MHL', 'FSM', 
    'PNG', 'STP', 'SLB', 'TLS', 'TUV', 
    'VEN', 'ZWE'
]

# Normalize country names for conflict and fragile lists
conflict_iso3 = [normalize_country_name(country) for country in CONFLICT_COUNTRIES]
fragile_iso3 = [normalize_country_name(country) for country in FRAGILE_COUNTRIES]

# Remove None values
conflict_iso3 = [iso3 for iso3 in conflict_iso3 if iso3]
fragile_iso3 = [iso3 for iso3 in fragile_iso3 if iso3]

# Add 'FCS', 'FCS_conflict', and 'FCS_fragile' columns to the grouped data
grouped_data['FCS'] = grouped_data['country_iso3'].isin(conflict_iso3 + fragile_iso3)
grouped_data['FCS_conflict'] = grouped_data['country_iso3'].isin(conflict_iso3)
grouped_data['FCS_fragile'] = grouped_data['country_iso3'].isin(fragile_iso3)

# Merge the org grouped data with the main grouped data
final_data = pd.merge(grouped_data, org_grouped_data, on='country_iso3')

# Save the aggregated data to a new CSV file
AGGREGATED_FILE_PATH = r"D:\data\CGIAR\UNFCCC\aggregated_master.csv"
final_data.to_csv(AGGREGATED_FILE_PATH, index=False, encoding='utf-8')

final_data.head()


,country_iso3,grant_amount,population,grant_per_capita,FCS,FCS_conflict,FCS_fragile,AF,GCF,GEF,grant_per_capita_AF,grant_per_capita_GCF,grant_per_capita_GEF
0,AFG,7.523781e+07,41128771,1.829323,True,True,False,0.0,1.719884e+07,5.803897e+07,0.0,0.418171,1.411153
1,AGO,8.828583e+07,35588987,2.480706,False,False,False,11941038.0,0.000000e+00,7.634479e+07,0.335526,0.0,2.14518
2,ALB,6.346315e+07,2777689,22.847464,False,False,False,9927750.0,3.578221e+07,1.775319e+07,3.574104,12.882006,6.391354
3,ARG,3.632480e+08,46234830,7.856587,False,False,False,23936813.0,2.081240e+08,1.311871e+08,0.517723,4.501456,2.837409
4,ARM,1.770356e+08,2780469,63.67114,False,False,False,3941100.0,1.393975e+08,3.369706e+07,1.417423,50.134519,12.119198


comparison with GCF data

In [60]:
# Load the GCF export file
gcf_file_path = r"D:\data\CGIAR\UNFCCC\GCF-Export-countries-1716912441595.xlsx" #use file exported with country level financing column FA Financing the other is for readiness grants. 
gcf_data = pd.read_excel(gcf_file_path)

# Merge the GCF data with the final_data on country_iso3
comparison_data = pd.merge(final_data, gcf_data[['ISO3', 'FA Financing $']], left_on='country_iso3', right_on='ISO3', how='inner')

# Calculate the error percentage
comparison_data['error_percentage'] = ((comparison_data['GCF'] - comparison_data['FA Financing $']) / comparison_data['FA Financing $']) * 100

# Save the comparison data to a new CSV file
comparison_file_path = r"D:\data\CGIAR\UNFCCC\comparison_data.csv"
comparison_data.to_csv(comparison_file_path, index=False, encoding='utf-8')

# Display the comparison data
print(comparison_data.head())


  country_iso3  grant_amount population grant_per_capita    FCS  FCS_conflict  \
0          AFG  7.523781e+07   41128771         1.829323   True          True   
1          AGO  8.828583e+07   35588987         2.480706  False         False   
2          ALB  6.346315e+07    2777689        22.847464  False         False   
3          ARG  3.632480e+08   46234830         7.856587  False         False   
4          ARM  1.770356e+08    2780469         63.67114  False         False   

   FCS_fragile          AF           GCF           GEF grant_per_capita_AF  \
0        False         0.0  1.719884e+07  5.803897e+07                 0.0   
1        False  11941038.0  0.000000e+00  7.634479e+07            0.335526   
2        False   9927750.0  3.578221e+07  1.775319e+07            3.574104   
3        False  23936813.0  2.081240e+08  1.311871e+08            0.517723   
4        False   3941100.0  1.393975e+08  3.369706e+07            1.417423   

  grant_per_capita_GCF grant_per_capita_GEF 